In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from scipy.sparse.linalg import eigsh
from scipy import sparse
from numpy import genfromtxt

# initialize variables
N = 10 # number of sites
half_filling = True

if half_filling == True:

    N_e = N # total number of electrons
    S_z = 0
W = 2*N # each sites can have 2 electrons
# Q is the dimension
Q = pow(4, N)

# decimal number to binary array function
def D2B(num):
    string = f'{num:1b}'
    result = np.zeros(W-len(string), int)

    for ele in string:
        result = np.append(result, int(ele))

    return result #nd array


# binary array to decimal function
def B2D(array):
    res = 0
    for ele in array:
        res = (res << 1) | ele
    return res[0]

In [ ]:
half_N = N // 2  # Number of columns

# Create a list of tuples representing the grid labels
labels = [(row, col) for row in range(1, 3) for col in range(1, half_N + 1)]

# Create a dictionary to map each label to a unique number starting from 0
label_to_number = {label: index for index, label in enumerate(labels)}

# Reverse the dictionary: keys become values and values become keys
number_to_label = {v: k for k, v in label_to_number.items()}

In [ ]:
number_to_label

{0: (1, 1),
 1: (1, 2),
 2: (1, 3),
 3: (1, 4),
 4: (1, 5),
 5: (2, 1),
 6: (2, 2),
 7: (2, 3),
 8: (2, 4),
 9: (2, 5)}

In [ ]:
def are_nearest_neighbors(p1, p2):
    """
    Check if two tuples representing grid coordinates are nearest neighbors.

    Parameters:
    p1 (tuple): The first coordinate (i, j).
    p2 (tuple): The second coordinate (m, l).

    Returns:
    str: "Yes" if p1 and p2 are nearest neighbors, "No" otherwise.
    """
    (i, j) = p1
    (m, l) = p2

    # Check if p1 and p2 are nearest neighbors
    row_diff = abs(i - m)
    col_diff = abs(j - l)

    # Nearest neighbors if exactly one of the differences is 1 and the other is 0
    if (row_diff == 1 and col_diff == 0) or (row_diff == 0 and col_diff == 1):
        return True
    else:
        return False

In [ ]:
# Initialize t_matrix
t_matrix = np.zeros((N, N), dtype='float64')

# Hopping value
t_value = -1  # Nearest-neighbor (NN) hopping

# Define periodic boundary condition (PBC) along rows
PBC_condition = True

for i in range(N):
    if PBC_condition == True:
        t_matrix[0][N//2-1] = t_value # the first row start and end value
        t_matrix[N//2][N-1] = t_value # the second row start and end value this is enough it is always 2x N_x here
    for j in range(N):
        # now check if NN
        if j >i:
            if are_nearest_neighbors(number_to_label[i], number_to_label[j]):
                t_matrix[i][j] = t_value

# add another half back
t_matrix = np.transpose(t_matrix) + t_matrix
t_matrix

array([[ 0., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.],
       [-1.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0., -1.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0., -1.,  0., -1.,  0.,  0.,  0., -1.,  0.],
       [-1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -1.],
       [-1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.],
       [ 0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1., -1.,  0.,  0., -1.,  0.]])

In [ ]:
# load  data
file_path = f'/content/drive/My Drive/Hubbard_2024/2d/old_index_list={N}.txt'
old_index_list = np.loadtxt(file_path, dtype=int)

In [ ]:
# N_e list getting a list with correct numberr of electrcons without sz limitation

# create dict for new index in N_e sector

NE_list=[]
for i in range(Q):
    state = D2B(i)
    up_total=0
    down_total=0
    state_4_U = state.reshape((N, 2))
    for j in range(N):
        up_total = up_total + state_4_U[j][0]
        down_total = down_total + state_4_U[j][1]
    if (np.sum(state) == N_e):
        NE_list.append(i)

NE_dimension  = len(NE_list)
print(NE_dimension)

184756


In [ ]:
# The U part does not need matrix to do calculation it just attach a phase to the state vector later

In [ ]:
unit_U_matrix_row_list=[] # diagonal matrix anyways hence row is col as well, dimension is len( old_index_list )
double_occ_site_index_list=[]
for index, value in enumerate(old_index_list):

    state = D2B(value)
    # U part
    # reshape it so that each array for one electron
    state_4_U = state.reshape((N, 2))
    # loop each array to see if how many double occuplied
    number_of_double_occ_list = []
    for j in range(N):
        one_electron_array = state_4_U[j]
        one_electron_array_sum = np.sum(one_electron_array)
        if one_electron_array_sum ==2:
            number_of_double_occ_list.append(j)
    if len(number_of_double_occ_list)!=0:
        double_occ_site_index_list.append(number_of_double_occ_list)
        unit_U_matrix_row_list.append(index)

In [ ]:
# explain : unit_U_matrix_row_list is the list that gives the index of elements in old_index_list
# that does not have double occ
# For example, in N=4, it does not have 11 or 14,
# bc D2B(old_index_list[11]) gives [0, 1, 0, 1, 1, 0, 1, 0]
# D2B(old_index_list[14]) gives  [0, 1, 1, 0, 1, 0, 0, 1]
# double_occ_site_index_list is the site index for that state, which site has double occ

In [ ]:
def get_sprase_U_matrix(input_array): #input array is the U value for each site
    row = unit_U_matrix_row_list
    col = unit_U_matrix_row_list

    # get U value, number of double occ times the corresponding U value
    data = np.zeros(len(row))
    for index in range(len(double_occ_site_index_list)):
        value = double_occ_site_index_list[index]
        total_U_for_one_state = 0
        for site_index in value:
            total_U_for_one_state = total_U_for_one_state + input_array[site_index]
        data[index] = total_U_for_one_state

    sprase_U_matrix = sparse.coo_matrix((data, (row, col)), shape=(len(old_index_list), len(old_index_list) ) ) # len of old is full dimension
    # row col is the ones with non zero U value

    return sprase_U_matrix


In [ ]:
# this cell is getting all the possible hopping and put it into a product state basis matrix and to fill in the hopping strength later
# can be limited to t matrix if N is too big to get all the possible hopping case
new_index_list=np.arange(0,len(old_index_list))
index_dict = dict(zip(old_index_list, new_index_list))

unit_hop_matrix_row_list=[]
unit_hop_matrix_col_list=[]
unit_hop_matrix_data_list=[]
data_site_row_list =[]
data_site_col_list =[]

# start looping each state and see what other states it couple to
for index, value in enumerate(old_index_list):

    state = D2B(value)
    # reshape it so that each array for one electron
    state_4_U = state.reshape((N, 2))

    # t part

    # where it starts to hop
    for j in range(N):
        # loop where it hops to
        for k in range(N):

            # up spin part  that is why [0]
            if state_4_U[j][0]==1 and state_4_U[k][0]==0:
                number_pass = 0
                hop_state = np.copy(state_4_U) # a copy of the state so that it won't change the original one
                state_to_count = np.copy(state)
                start_position = 2*j
                # count how many from the edge to start position
                number_pass = number_pass + np.sum(state_to_count[:start_position])
                # remove that one
                state_to_count[start_position] = 0
                # count how many from the edge to end position
                end_position = 2*k
                number_pass = number_pass + np.sum(state_to_count[:end_position])
                hop_state[j][0] =0
                hop_state[k][0] =1
                # new number
                hop_state = hop_state.reshape((2*N, 1))
                new_num = B2D(hop_state)
                # for hopping matrix

                unit_hop_matrix_row_list.append(index)
                unit_hop_matrix_col_list.append(index_dict[new_num])
                unit_hop_matrix_data_list.append(pow(-1,number_pass))
                data_site_row_list.append(j)
                data_site_col_list.append(k)


            # down spin part
            if state_4_U[j][1]==1 and state_4_U[k][1]==0:
                number_pass = 0
                hop_state = np.copy(state_4_U) # a copy of the real state so that it won't change the original one
                state_to_count = np.copy(state)
                start_position = 2*j+1
                # count how many from the edge to start position
                number_pass = number_pass + np.sum(state_to_count[:start_position])
                # remove that one
                state_to_count[start_position] = 0
                # count how many from the edge to end position
                end_position = 2*k+1
                number_pass = number_pass + np.sum(state_to_count[:end_position])
                hop_state[j][1] =0
                hop_state[k][1] =1
                # new number
                hop_state = hop_state.reshape((2*N, 1))
                new_num = B2D(hop_state)
                # for hopping matrix
                unit_hop_matrix_row_list.append(index)
                unit_hop_matrix_col_list.append(index_dict[new_num])
                unit_hop_matrix_data_list.append(pow(-1,number_pass))
                data_site_row_list.append(j)
                data_site_col_list.append(k)

unit_hop_matrix_row_list= np.array (unit_hop_matrix_row_list)
unit_hop_matrix_col_list= np.array (unit_hop_matrix_col_list)
unit_hop_matrix_data_list= np.array (unit_hop_matrix_data_list)
data_site_row_list = np.array(data_site_row_list)
data_site_col_list = np.array(data_site_col_list)

In [ ]:
# get sprase H_hop
def get_sprase_hop_matrix(t_matrix_input): #input array is the U value for each site


    hop_matrix_data_list = []

    for index, value in enumerate(unit_hop_matrix_data_list):

        site_index_row = data_site_row_list[index]
        site_index_col = data_site_col_list[index]

        new_value = value * t_matrix_input[site_index_row][site_index_col] # value is either 1 or -1 and times the strength at the corresponding site

        hop_matrix_data_list.append(new_value)
    hop_matrix_data_list =np.array(hop_matrix_data_list)
    # hop_matrix_data_list is the data later is the dimension for where to fill and then is the full dimension
    sprase_hop_matrix_1 = sparse.coo_matrix( ( hop_matrix_data_list, (unit_hop_matrix_row_list, unit_hop_matrix_col_list) )
                                          , shape=(len(old_index_list), len(old_index_list) ) ) # len of old is full dimension

    return sprase_hop_matrix_1

In [ ]:
# def fidelity(state1, state2):
#     return np.abs(np.vdot(state1, state2))**2

# # List to store ground states
# ground_states = []

# # Generate ground states for different U values
# U_value_list = np.array([0,2,4,6,8,10,12,14,16,18,20])
# # U_value_list = np.array([0,4,8,12])
# # U_value_list = np.array([0,8])
# for U_value in U_value_list:
#     h_sparse = get_sprase_hop_matrix(t_matrix) + get_sprase_U_matrix(np.ones(N)*U_value)
#     result = sparse.linalg.eigsh(h_sparse, k=1, which='SA')
#     true_gs = result[1][:, 0]  # Extract the ground state (eigenvector)
#     ground_states.append(true_gs)

# # Calculate fidelity between consecutive ground states
# fidelities = []
# for i in range(len(ground_states) - 1):
#     fid = fidelity(ground_states[i], ground_states[i+1])
#     fidelities.append(fid)
#     print(f"Fidelity between U={U_value_list[i]} and U={U_value_list[i+1]}: {fid}")


In [ ]:
def s2_op(input_state):
    #calculating S+S- only, c.c. gives S-S+. Also, phi S+S- psi can be simplified to just S- onto psi to know the scaling factor,and Sz is easy to get,assuming 0 or 1
    # lowering part
    new_state = np.zeros(len(NE_list))
    for index,value in enumerate(old_index_list): # loop over every computational basis
        state = D2B(value)
        # reshape it so that each array for one electron
        state_4_U = state.reshape((N, 2))
        for j in range(N):
            hop_state = np.copy(state_4_U) # a copy of the real state so that it won't change the original one
            # up spin part
            if (state_4_U[j][0]==1) and (state_4_U[j][1]==0):
                # same site all the time hence it's sqaure of the either -1 or 1 but both give 1
                hop_state[j][0] = 0
                hop_state[j][1] = 1
                # new number
                hop_state = hop_state.reshape((2*N, 1))
                new_num = B2D(hop_state)
                #print(new_num)
                # Correct usage of index method
                try:
                    new_index = NE_list.index(new_num)
                    new_state[new_index] +=  input_state[index]  # Update new_state using the scaling factor
                except ValueError:
                    # Handle the case if new_num is not found in NE_list
                    print(f"Value {new_num} not found in NE_list.")
    value1 = np.round( pow(np.linalg.norm(new_state),2),7)
    # raising part
    new_state = np.zeros(len(NE_list))
    for index,value in enumerate(old_index_list): # loop over every computational basis
        state = D2B(value)
        # reshape it so that each array for one electron
        state_4_U = state.reshape((N, 2))
        for j in range(N):
            hop_state = np.copy(state_4_U) # a copy of the real state so that it won't change the original one
            # up spin part
            if (state_4_U[j][0]==0) and (state_4_U[j][1]==1):
                # same site all the time hence it's sqaure of the either -1 or 1 but both give 1
                hop_state[j][0] = 1
                hop_state[j][1] = 0
                # new number
                hop_state = hop_state.reshape((2*N, 1))
                new_num = B2D(hop_state)
                #print(new_num)
                # Correct usage of index method
                try:
                    new_index = NE_list.index(new_num)
                    new_state[new_index] +=  input_state[index]  # Update new_state using the scaling factor
                except ValueError:
                    # Handle the case if new_num is not found in NE_list
                    print(f"Value {new_num} not found in NE_list.")
    value2 = np.round( pow(np.linalg.norm(new_state),2),7)

    return (value1+value2)/2   # this is S(S+1) value without Sz^2

In [ ]:
# read def spin matrix
# Define the path in Google Drive
file_path = f'/content/drive/My Drive/Hubbard_2024/2d/definite_spin_matrix_N={N}.npz'

# Read the sparse matrix
spin_matrix = sparse.load_npz(file_path)

def quantum_expectation(sparse_matrix, phi):
    """
    Calculates the quantum mechanical expectation value of the form phi^H * matrix * phi.

    Args:
      sparse_matrix: A sparse matrix (e.g., scipy.sparse.csr_matrix).
      phi: A NumPy array representing the state vector.

    Returns:
      The scalar expectation value of the form phi^H * matrix * phi, rounded to 7 decimal places.
    """
    # Step 1: Take the conjugate transpose of phi
    phi_conj = np.conjugate(phi)

    # Step 2: Multiply the sparse matrix with phi to get an intermediate result
    intermediate_result = sparse_matrix.dot(phi)

    # Step 3: Compute the final dot product (phi_conj * intermediate_result)
    result = phi_conj @ intermediate_result  # Using @ for dot product for better readability

    # Step 4: Return the result rounded to 7 decimal places
    return np.round(result, 7)


In [ ]:
U_value_list = np.array([1,2,4,6,8,16])
summary_data = []
for U_value in U_value_list:
    number_of_excited_state = 10
    h_sprase = get_sprase_hop_matrix(t_matrix) + get_sprase_U_matrix(np.ones(N)*U_value)
    sparse.save_npz(f'/content/drive/My Drive/Hubbard_2024/2d/es/torch_h_sparse_N_{N}_U_{U_value}.npz', h_sprase.tocoo())
    result = sparse.linalg.eigsh(h_sprase, k=10, which='SA')
    true_gs = result[1].T[0]

    # also serve as a check
    S_value_gs = quantum_expectation(spin_matrix,true_gs)
    # or use general method
    #S_value_gs = s2_op(true_gs)
    print('S_value_gs is',S_value_gs)
    # now find the same S first couple excited state

    for state_index in range(1,number_of_excited_state):
        exctied_state = result[1].T[state_index]

        # get S value
        S_value = quantum_expectation(spin_matrix,exctied_state)
        #S_value = s2_op(exctied_state)
        print(S_value)
        if S_value == S_value_gs:
            first_excited_state = exctied_state # bc we are looking for the lowest enregy one, automatically handled by the ranking of the vector already
            print('state_index is ', state_index,'S_value is ',S_value )
            break

    # save result
    np.savetxt(f'/content/drive/My Drive/Hubbard_2024/2d/es/true_gs_energy_N_{N}_U_{U_value}.csv', [result[0][0]], delimiter=',')
    np.savetxt(f'/content/drive/My Drive/Hubbard_2024/2d/es/first_couple_excited_energy_N_{N}_U_{U_value}.csv', [result[0][state_index]], delimiter=',')
    np.savetxt(f'/content/drive/My Drive/Hubbard_2024/2d/es/true_gs_N_{N}_U_{U_value}.csv', true_gs, delimiter=',')
    np.savetxt(f'/content/drive/My Drive/Hubbard_2024/2d/es/first_couple_excited_state_N_{N}_U_{U_value}_Ne.csv', first_excited_state, delimiter=',')
    np.savetxt(f'/content/drive/My Drive/Hubbard_2024/2d/es/first_excited_energy_N_{N}_U_{U_value}.csv', [result[0][1]], delimiter=',')
    np.savetxt(f'/content/drive/My Drive/Hubbard_2024/2d/es/first_excited_state_N_{N}_U_{U_value}.csv', first_excited_state, delimiter=',')

    # Add to summary data
    summary_data.append({
        "U_value": U_value,
        "gs_energy": result[0][0],
        "first_couple_excited_energy": result[0][state_index],
        "first_excited_energy": result[0][1]
    })

# Print summary of energies at the end
print("\nSummary of energies for different U values:")
for data in summary_data:
    print(f"U = {data['U_value']}: GS Energy = {data['gs_energy']}, First Couple Excited Energy = {data['first_couple_excited_energy']}, First Excited Energy = {data['first_excited_energy']}")

In [ ]:
# N = 8
# S_value_gs is -0.0
# 2.0
# 2.0
# 2.0
# 2.0
# -0.0
# state_index is  5 S_value is  -0.0
# S_value_gs is -0.0
# 2.0
# 2.0
# 2.0
# 2.0
# 0.0
# state_index is  5 S_value is  0.0
# S_value_gs is -0.0
# 2.0
# 2.0
# 2.0
# 2.0
# 0.0
# state_index is  5 S_value is  0.0
# S_value_gs is 0.0
# 2.0
# 2.0
# 2.0
# 2.0
# 6.0
# 2.0
# 2.0
# 2.0
# -0.0
# state_index is  9 S_value is  -0.0
# S_value_gs is -0.0
# 2.0
# 2.0
# 2.0
# 2.0
# 6.0
# 2.0
# 2.0
# 2.0
# -0.0
# state_index is  9 S_value is  -0.0
# S_value_gs is 0.0
# 2.0
# 2.0
# 2.0
# 2.0
# 2.0
# 2.0
# 2.0
# 6.0
# 0.0
# state_index is  9 S_value is  0.0

# Summary of energies for different U values:
# U = 1: GS Energy = -10.118762934506028, First Couple Excited Energy = -8.145959053711838, First Excited Energy = -8.491884337534497
# U = 2: GS Energy = -8.478303296869594, First Couple Excited Energy = -6.5857864444709655, First Excited Energy = -7.206741230016821
# U = 4: GS Energy = -5.954236681057318, First Couple Excited Energy = -4.356277445472863, First Excited Energy = -5.256982547718461
# U = 8: GS Energy = -3.4671734372139245, First Couple Excited Energy = -2.4370455114703824, First Excited Energy = -3.1753036376269583
# U = 16: GS Energy = -1.8772690847081202, First Couple Excited Energy = -1.3136244826137233, First Excited Energy = -1.701812766199699

In [ ]:
# S_value_gs is 2.0
# 0.0
# -0.0
# 0.0
# 2.0
# state_index is  4 S_value is  2.0
# S_value_gs is 2.0
# 0.0
# 0.0
# 0.0
# 2.0
# state_index is  4 S_value is  2.0
# S_value_gs is -0.0
# 2.0
# 2.0
# 2.0
# -0.0
# state_index is  4 S_value is  -0.0
# S_value_gs is -0.0
# 2.0
# 2.0
# 2.0
# -0.0
# state_index is  4 S_value is  -0.0
# S_value_gs is 0.0
# 2.0
# 2.0
# 2.0
# -0.0
# state_index is  4 S_value is  -0.0
# S_value_gs is -0.0
# 2.0
# 2.0
# 2.0
# -0.0
# state_index is  4 S_value is  -0.0

# Summary of energies for different U values:
# U = 1: GS Energy = -11.453171087300346, First Couple Excited Energy = -11.230390277938188, First Excited Energy = -11.3813361188669
# U = 2: GS Energy = -9.508902323907462, First Couple Excited Energy = -9.325412309894787, First Excited Energy = -9.47281389008344
# U = 4: GS Energy = -6.805335072522798, First Couple Excited Energy = -6.4635195428644066, First Excited Energy = -6.642914070897716
# U = 8: GS Energy = -4.1546970562649035, First Couple Excited Energy = -3.6828367397433164, First Excited Energy = -3.887124616402076
# U = 16: GS Energy = -2.2617602698499937, First Couple Excited Energy = -1.936516763551882, First Excited Energy = -2.0731243851171044

In [ ]:
# # check after multiplying definite spin

# #Define the path in Google Drive
# file_path = f'/content/drive/My Drive/Hubbard_2024/product_state/definite_spin_basis_N={N}.npz'
# #Save the sparse matrix
# definite_spin_basis= sparse.load_npz(file_path)

In [ ]:
# def_spin_h_sprase = definite_spin_basis*h_sprase*definite_spin_basis.T

In [ ]:
# sparse.linalg.eigsh(h_sprase, k=174, which='SA')[0]

In [ ]:
# sparse.linalg.eigsh(def_spin_h_sprase, k=10, which='SA')[0]

In [ ]:
# definite spin h is much more dense than the original H , use original H instead